In [26]:
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio as rio
import matplotlib.pyplot as plt

import os
from typing import Any

In [ ]:
# def fill_na(arr: np.typing.NDArray, no_val: int, fill_val: Any = np.nan) -> np.typing.NDArray:
#   copy = np.copy(arr)
#   copy[copy == no_val] = fill_val
#   return copy

In [83]:
area_names = ["croton_outer", "catskill_delaware_outer"]
collection_year_range = range(1985, 2025, 1)
# simply because I can't figure out a better way to do this >_<
decadal_stops = [1994, 2004, 2014, 2024]
file_type_dic = {
  "Land Cover": "LndCov",
  "Land Cover Change": "LndChg",
  "Land Cover Confidence": "LndCnf",
  "Fractional Impervious Surface": "FctImp",
  "Impervious Descriptor": "ImpDsc",
  "Spectral Change Day of Year": "SpcChg"
}
nlcd_annual_fname_template = "Annual_NLCD_{file_type}_{year}_CU_C1V1.tiff"
nlcd_annual_fractional_impervious_change_template = "Annual_NLCD_FctImp_{from_year}_{to_year}_CU_C1V1.tiff"

landcover_dir = os.path.relpath("../datasets/landcover")
nlcd_landcover_dir = os.path.join(landcover_dir, "nlcd_annual")
fractional_impervious_change_dir = os.path.join(landcover_dir, "fractional_impervious", "change")



In [65]:
valid_data_def_dict = {
  "Land Cover": {"valid_range": pd.Interval(closed="both", left=11, right=95), "no_data": 250},
  "Land Cover Change": {"valid_range": pd.Interval(closed="both", left=11, right=9590), "no_data": 9999},
  "Land Cover Confidence": {"valid_range": pd.Interval(closed="both", left=1, right=100), "no_data": 250},
  "Fractional Impervious Surface": {"valid_range": pd.Interval(closed="both", left=0, right=100), "no_data": 250},
  "Impervious Descriptor": {"valid_range": pd.Interval(closed="both", left=0, right=2), "no_data": 250},
  "Spectral Change Day of Year": {"valid_range": pd.Interval(closed="both", left=0, right=366), "no_data": 9999},
}

In [113]:
fname_test = os.path.join(nlcd_landcover_dir, "croton_outer", "Annual_NLCD_FctImp_2024_CU_C1V1.tiff")

In [114]:
with rio.open(fname_test) as src:
  fractional_imp_ashokan_85 = src.read(1).astype(np.float64)
  profile = src.meta.copy()

# plt.imshow(fractional_imp_ashokan_85)
profile

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': 250.0,
 'width': 2453,
 'height': 2588,
 'count': 1,
 'crs': CRS.from_wkt('PROJCS["AEA        WGS84",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'),
 'transform': Affine(30.0, 0.0, 1798455.0,
        0.0, -30.0, 2291385.0)}

In [79]:
valid_interval = valid_data_def_dict["Fractional Impervious Surface"]["valid_range"]
m = ((fractional_imp_ashokan_85 >= valid_interval.left) & (fractional_imp_ashokan_85 <= valid_interval.right))
valid_cells = m.sum()
fractional_impervious = fractional_imp_ashokan_85.sum(where=m)

# resolution is 30m2
valid_area = valid_cells*30*30
impervious_area = (fractional_impervious*30*30)/100
total_fractional_impervious = impervious_area/valid_area

print(f"valid area is {valid_area}. impervious area is {impervious_area}. impervious fraction is {total_fractional_impervious}")

valid area is 3261420900. impervious area is 226370124.0. impervious fraction is 0.06940843605926485


In [119]:
def write_change_raster(from_year: int, to_year: int, area_name: str) -> None:
  file_type_code = file_type_dic["Fractional Impervious Surface"]
  no_data_val = valid_data_def_dict["Fractional Impervious Surface"]["no_data"]
  fname_prev_year = nlcd_annual_fname_template.format(file_type=file_type_code, year=from_year)
  fname_curr_year = nlcd_annual_fname_template.format(file_type=file_type_code, year=to_year)
  out_fname = nlcd_annual_fractional_impervious_change_template.format(from_year=from_year, to_year=to_year)
  out_path = os.path.join(fractional_impervious_change_dir, area_name, out_fname)
  path_prev_year = os.path.join(nlcd_landcover_dir, area_name, fname_prev_year)
  path_curr_year = os.path.join(nlcd_landcover_dir, area_name, fname_curr_year)
  if (os.path.isfile(path_prev_year) and os.path.isfile(path_curr_year)) != True:
    print("missing one or more files:", path_prev_year, path_curr_year)
    return

  with rio.open(path_prev_year) as src:
    prev_yr_data = src.read(1).astype(np.int16)
    profile = src.meta.copy()
  
  with rio.open(path_curr_year) as src:
    curr_yr_data = src.read(1).astype(np.int16)
  
  # if either year has no data, set it to the nodata val for both years
  m = (prev_yr_data != no_data_val) & (curr_yr_data != no_data_val)
  squashed_prev_data = np.where(m, prev_yr_data, no_data_val)
  squashed_curr_data = np.where(m, curr_yr_data, no_data_val)
  
  change = squashed_curr_data - squashed_prev_data
  
  profile.update(dtype=rio.int16, nodata=0)
  with rio.open(out_path, "w", **profile) as out:
    out.write(change, 1)

In [120]:

for a_name in area_names:
  write_change_raster(from_year=1985, to_year=2024, area_name=a_name)
  # calculate yoy changes
  for year in range(1986, 2025, 1):
    write_change_raster(from_year=year-1, to_year=year, area_name=a_name)
    if year in decadal_stops:
      write_change_raster(from_year=year-9, to_year=year, area_name=a_name)